# TF-TRT Inference from TensorFlow 2 Detection Model 


## Introduction
This notebook demonstrates the process of creating and benchmarking a TF-TRT optimization model from a TensorFlow 2 Detection Model Zoo model.

## Requirement

### GPU

Before running this notebook, please set the Colab runtime environment to GPU via the menu *Runtime => Change runtime type => GPU*.

This demo will work on any NVIDIA GPU with CUDA cores, though for improved FP16 and INT8 inference, a Volta, Turing or newer generation GPU with Tensor cores is desired.  On Google Colab, this normally means a T4 GPU. If you are assigned an older K80 GPU, another trial at another time might give you a T4 GPU.

In [1]:
!nvidia-smi

Sun Apr 18 23:38:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1070    Off  | 00000000:09:00.0  On |                  N/A |
|  0%   42C    P8    13W / 151W |    332MiB /  8116MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 23
model		: 1
model name	: AMD Ryzen 7 1700 Eight-Core Processor
stepping	: 1
microcode	: 0x8001138
cpu MHz		: 1550.000
cache size	: 512 KB
physical id	: 0
siblings	: 16
core id		: 0
cpu cores	: 8
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid aperfmperf pni pclmulqdq monitor ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand lahf_lm cmp_legacy svm extapic cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw skinit wdt tce topoext perfctr_core perfctr_nb bpext perfctr_llc mwaitx cpb hw_pstate sme ssbd sev ibpb vmmcall fsgsbase bmi1 avx2 smep bmi2 rdseed adx smap clflushopt sha_ni xsaveopt xsavec xgetbv1 xsaves clzero irperf xsaveerptr arat npt lbrv svm_lock nrip_save ts

In [3]:
!cat /proc/meminfo

MemTotal:       32868824 kB
MemFree:        12567568 kB
MemAvailable:   26614080 kB
Buffers:          592032 kB
Cached:         12983860 kB
SwapCached:            0 kB
Active:          2351016 kB
Inactive:       16185900 kB
Active(anon):      14780 kB
Inactive(anon):  5207668 kB
Active(file):    2336236 kB
Inactive(file): 10978232 kB
Unevictable:        4732 kB
Mlocked:            4732 kB
SwapTotal:      17596408 kB
SwapFree:       17596408 kB
Dirty:               432 kB
Writeback:             0 kB
AnonPages:       4965568 kB
Mapped:           955080 kB
Shmem:            262528 kB
KReclaimable:    1195900 kB
Slab:            1454924 kB
SReclaimable:    1195900 kB
SUnreclaim:       259024 kB
KernelStack:       17696 kB
PageTables:        53788 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    34030820 kB
Committed_AS:   13386636 kB
VmallocTotal:   34359738367 kB
VmallocUsed:      102368 kB
VmallocChunk:          0 kB
Percpu:          

### Check the version of TensorFlow-GPU.

In [4]:
# check TensorRT version
print("TensorRT version: ")
!dpkg -l | grep nvinfer

TensorRT version: 
/bin/bash: dpkg: コマンドが見つかりません


In [5]:
import tensorflow as tf
from tensorflow.compiler.tf2tensorrt._pywrap_py_utils import is_tensorrt_enabled
 
print("Tensorflow version: ", tf.version.VERSION)
print("TensorRT enabled  : ", is_tensorrt_enabled())

Tensorflow version:  2.4.1
TensorRT enabled  :  False


### Check Tensor core GPU
The below code check whether a Tensor-core GPU is present.

In [6]:
from tensorflow.python.client import device_lib
 
def check_tensor_core_gpu_present():
    local_device_protos = device_lib.list_local_devices()
    for line in local_device_protos:
        if "compute capability" in str(line):
            compute_capability = float(line.physical_device_desc.split("compute capability: ")[-1])
            if compute_capability>=7.0:
                return True
    
print("Tensor Core GPU Present:", check_tensor_core_gpu_present())
tensor_core_gpu = check_tensor_core_gpu_present()

Tensor Core GPU Present: None


### Importing required libraries

In [7]:
import os
import csv
import shutil

### Download COCO Val 2017 Dataset.

In [8]:
if not os.path.exists("./val2017") and not os.path.exists("./annotations"):
    !wget http://images.cocodataset.org/zips/val2017.zip
    !unzip -q val2017.zip
    !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip -q annotations_trainval2017.zip
    !rm -r val2017.zip annotations_trainval2017.zip

### Models

In [9]:
MODELS = {
    "centernet_hg104_512x512_coco17_tpu": {
        "name": "CenterNet\ HourGlass104",
        "input": (512, 512),
        "extract_dir": "centernet_hg104_512x512_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz",
    },
    "centernet_hg104_512x512_kpts_coco17_tpu": {
        "name": "CenterNet\ HourGlass104\ Keypoints",
        "input": (512, 512),
        "extract_dir": "centernet_hg104_512x512_kpts_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz",
    },
    "centernet_hg104_1024x1024_coco17_tpu": {
        "name": "CenterNet\ HourGlass104",
        "input": (1024, 1024),
        "extract_dir": "centernet_hg104_1024x1024_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz",
    },
    "centernet_hg104_1024x1024_kpts_coco17_tpu": {
        "name": "CenterNet\ HourGlass104\ Keypoints",
        "input": (1024, 1024),
        "extract_dir": "centernet_hg104_1024x1024_kpts_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_1024x1024_kpts_coco17_tpu-32.tar.gz",
    },
    "centernet_resnet50_v1_fpn_512x512_coco17_tpu": {
        "name": "CenterNet\ Resnet50\ V1\ FPN",
        "input": (512, 512),
        "extract_dir": "centernet_resnet50_v1_fpn_512x512_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.tar.gz",
    },
    "centernet_resnet50_v1_fpn_512x512_kpts_coco17_tpu": {
        "name": "CenterNet\ Resnet50\ V1\ FPN\ Keypoints",
        "input": (512, 512),
        "extract_dir": "centernet_resnet50_v1_fpn_512x512_kpts_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_kpts_coco17_tpu-8.tar.gz",
    },
    "centernet_resnet101_v1_fpn_512x512_coco17_tpu": {
        "name": "CenterNet\ Resnet101\ V1\ FPN",
        "input": (512, 512),
        "extract_dir": "centernet_resnet101_v1_fpn_512x512_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet101_v1_fpn_512x512_coco17_tpu-8.tar.gz",
    },
    "centernet_resnet50_v2_512x512_coco17_tpu": {
        "name": "CenterNet\ Resnet50\ V2",
        "input": (512, 512),
        "extract_dir": "centernet_resnet50_v2_512x512_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v2_512x512_coco17_tpu-8.tar.gz",
    },
    "centernet_resnet50_v2_512x512_kpts_coco17_tpu": {
        "name": "CenterNet\ Resnet50\ V2\ Keypoints",
        "input": (512, 512),
        "extract_dir": "centernet_resnet50_v2_512x512_kpts_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v2_512x512_kpts_coco17_tpu-8.tar.gz",
    },
    "centernet_mobilenetv2fpn_512x512_coco17_od": {
        "name": "CenterNet\ MobileNetV2\ FPN",
        "input": (512, 512),
        "extract_dir": "centernet_mobilenetv2_fpn_od",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz",
    },
    "centernet_mobilenetv2fpn_512x512_coco17_kpts": {
        "name": "CenterNet\ MobileNetV2\ FPN\ Keypoints",
        "input": (512, 512),
        "extract_dir": "centernet_mobilenetv2_fpn_kpts",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_kpts.tar.gz",
    },
    "efficientdet_d0_coco17_tpu": {
        "name": "EfficientDet\ D0",
        "input": (512, 512),
        "extract_dir": "efficientdet_d0_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz",
    },
    "efficientdet_d1_coco17_tpu": {
        "name": "EfficientDet\ D1",
        "input": (640, 640),
        "extract_dir": "efficientdet_d1_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz",
    },
    "efficientdet_d2_coco17_tpu": {
        "name": "EfficientDet\ D2",
        "input": (768, 768),
        "extract_dir": "efficientdet_d2_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d2_coco17_tpu-32.tar.gz",
    },
    "efficientdet_d3_coco17_tpu": {
        "name": "EfficientDet\ D3",
        "input": (896, 896),
        "extract_dir": "efficientdet_d3_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d3_coco17_tpu-32.tar.gz",
    },
    "efficientdet_d4_coco17_tpu": {
        "name": "EfficientDet\ D4",
        "input": (1024, 1024),
        "extract_dir": "efficientdet_d4_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz",
    },
    "efficientdet_d5_coco17_tpu": {
        "name": "EfficientDet\ D5",
        "input": (1280, 1280),
        "extract_dir": "efficientdet_d5_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d5_coco17_tpu-32.tar.gz",
    },
    "efficientdet_d6_coco17_tpu": {
        "name": "EfficientDet\ D6",
        "input": (1280, 1280),
        "extract_dir": "efficientdet_d6_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d6_coco17_tpu-32.tar.gz",
    },
    "efficientdet_d7_coco17_tpu": {
        "name": "EfficientDet\ D7",
        "input": (1536, 1536),
        "extract_dir": "efficientdet_d7_coco17_tpu-32",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d7_coco17_tpu-32.tar.gz",
    },
    "ssd_mobilenet_v2_320x320_coco17_tpu": {
        "name": "SSD\ MobileNet\ v2",
        "input": (320, 320),
        "extract_dir": "ssd_mobilenet_v2_320x320_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz",
    },
    "ssd_mobilenet_v1_fpn_640x640_coco17_tpu": {
        "name": "SSD\ MobileNet\ V1\ FPN",
        "input": (640, 640),
        "extract_dir": "ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz",
    },
    "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu": {
        "name": "SSD\ MobileNet\ V2\ FPNLite",
        "input": (320, 320),
        "extract_dir": "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz",
    },
    "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu": {
        "name": "SSD\ MobileNet\ V2\ FPNLite",
        "input": (640, 640),
        "extract_dir": "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz",
    },
    "ssd_resnet50_v1_fpn_640x640_coco17_tpu": {
        "name": "SSD\ ResNet50\ V1\ FPN\ RetinaNet50",
        "input": (640, 640),
        "extract_dir": "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz",
    },
    "ssd_resnet50_v1_fpn_1024x1024_coco17_tpu": {
        "name": "SSD\ ResNet50\ V1\ FPN\ RetinaNet50", 
        "input": (1024, 1024), 
        "extract_dir": "ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz",
    },
    "ssd_resnet101_v1_fpn_640x640_coco17_tpu": {
        "name": "SSD\ ResNet101\ V1\ FPN\ RetinaNet101",
        "input": (640, 640),
        "extract_dir": "ssd_resnet101_v1_fpn_640x640_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz",
    },
    "ssd_resnet101_v1_fpn_1024x1024_coco17_tpu": {
        "name": "SSD\ ResNet101\ V1\ FPN\ RetinaNet101",
        "input": (1024, 1024),
        "extract_dir": "ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.tar.gz",
    },
    "ssd_resnet152_v1_fpn_640x640_coco17_tpu": {
        "name": "SSD\ ResNet152\ V1\ FPN\ RetinaNet152",
        "input": (640, 640),
        "extract_dir": "ssd_resnet152_v1_fpn_640x640_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.tar.gz",
    },
    "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu": {
        "name": "SSD\ ResNet152\ V1\ FPN\ RetinaNet152",
        "input": (1024, 1024),
        "extract_dir": "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz",
    },
    "faster_rcnn_resnet50_v1_640x640_coco17_tpu": {
        "name": "Faster\ R-CNN\ ResNet50\ V1",
        "input": (640, 640),
        "extract_dir": "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz",
    },
    "faster_rcnn_resnet50_v1_1024x1024_coco17_tpu": {
        "name": "Faster\ R-CNN\ ResNet50\ V1",
        "input": (1024, 1024),
        "extract_dir": "faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8.tar.gz",
    },
    "faster_rcnn_resnet50_v1_800x1333_coco17_gpu": {
        "name": "Faster\ R-CNN\ ResNet50\ V1",
        "input": (800, 1333),
        "extract_dir": "faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8.tar.gz",
    },
    "faster_rcnn_resnet101_v1_640x640_coco17_tpu": {
        "name": "Faster\ R-CNN\ ResNet101\ V1",
        "input": (640, 640),
        "extract_dir": "faster_rcnn_resnet101_v1_640x640_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz",
    },
    "faster_rcnn_resnet101_v1_1024x1024_coco17_tpu": {
        "name": "Faster\ R-CNN\ ResNet101\ V1",
        "input": (1024, 1024),
        "extract_dir": "faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz",
    },
    "faster_rcnn_resnet101_v1_800x1333_coco17_gpu": {
        "name": "Faster\ R-CNN\ ResNet101\ V1",
        "input": (800, 1333),
        "extract_dir": "faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.tar.gz",
    },
    "faster_rcnn_resnet152_v1_640x640_coco17_tpu": {
        "name": "Faster\ R-CNN\ ResNet152\ V1",
        "input": (640, 640),
        "extract_dir": "faster_rcnn_resnet152_v1_640x640_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz",
    },
    "faster_rcnn_resnet152_v1_1024x1024_coco17_tpu": {
        "name": "Faster\ R-CNN\ ResNet152\ V1",
        "input": (1024, 1024),
        "extract_dir": "faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8.tar.gz",
    },
    "faster_rcnn_resnet152_v1_800x1333_coco17_gpu": {
        "name": "Faster\ R-CNN\ ResNet152\ V1",
        "input": (800, 1333),
        "extract_dir": "faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8.tar.gz",
    },
    "faster_rcnn_inception_resnet_v2_640x640_coco17_tpu": {
        "name": "Faster\ R-CNN\ Inception\ ResNet\ V2",
        "input": (640, 640),
        "extract_dir": "faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz",
    },
    "faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu": {
        "name": "Faster\ R-CNN\ Inception\ ResNet\ V2",
        "input": (1024, 1024),
        "extract_dir": "faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz",
    },
    "mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu": {
        "name": "Mask\ R-CNN\ Inception\ ResNet\ V2",
        "input": (1024, 1024),
        "extract_dir": "mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8",
        "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz",
    },
    # extremernet
    # "extremenet": {
    #     "name": "ExtremeNet",
    #     "input": (, ),
    #     "extract_dir": "extremenet",
    #     "url": "http://download.tensorflow.org/models/object_detection/tf2/20200711/extremenet.tar.gz",
    # },
}

In [10]:
output_path = "./result.csv"

In [11]:
def num_record(path):
    return sum([1 for _ in open(path)])

In [12]:
def output_failed_result(path, name, input, tftrt):
    faild_result = ["", "", "", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    faild_result[0] = name
    faild_result[1] = input
    faild_result[2] = tftrt
    with open(path, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(faild_result)

In [13]:
def is_record(path, model_name, width, height):
    result = False
    model = model_name.replace('\\', '')
    size = str(height) + "x" + str(width)

    with open(path) as f:
        reader = csv.reader(f)
        for row in reader:
            if model == row[0].replace('\\', '') and size == row[1]:
                result = True
                break

    return result

In [14]:
def write_header(path):
    # write result csv header
    header = ["Model", "Input", "TF-TRT",
              "AP", "AP50", "AP75", "APsmall", "APmedium", "APlarge",
              "ARmax=1", "ARmax=10", "ARmax=100", "ARsmall", "ARmidium", "ARlarge",
              "images/sec", "99th_percentile(ms)", "total_time(s)", "latency_mean(ms)",
              "latency_median(ms)", "latency_min(ms)"]

    with open(path, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)

In [21]:
if not os.path.exists(output_path):
    print("Create result file.")
    write_header(output_path)

Create result file.


In [22]:
num_csv_record = 0
val2017_dir = "./val2017"
annotations_dir = "./annotations/instances_val2017.json"
input_batch_size = 1
display_every = 1000

for key in MODELS:
    print("start --------------------" + key + "--------------------")
    url = MODELS[key]["url"]
    extract_dir = MODELS[key]["extract_dir"]
    tar_file_name = url.rsplit("/", 1)[1]
    input_saved_model_dir = os.path.join(".", MODELS[key]["extract_dir"], "saved_model")
    output_dir = os.path.join(".", MODELS[key]["extract_dir"], "convert")
    input_height = MODELS[key]["input"][0]
    input_width = MODELS[key]["input"][1]
    max_workspace_size = 2 * 1024 * 1024 * 1024
    minimum_segment_size = 50
    model_name = MODELS[key]["name"]

    # check result
    if is_record(output_path, model_name, input_width, input_height):
        print("Already measured. Skip.")
        continue

    # download model and extrac file.
    if not os.path.exists(tar_file_name):
          !wget $url
    !tar xf $tar_file_name

    # benchmark original model.
    print("-------------------- Native FP32 model --------------------")

    before_record = num_record(output_path)
    output_saved_model_dir = os.path.join(output_dir, "Native")
    !TF_XLA_FLAGS="--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit" python object_detection.py \
        --input_saved_model_dir $input_saved_model_dir \
        --output_saved_model_dir $output_saved_model_dir \
        --data_dir $val2017_dir \
        --annotation_path $annotations_dir \
        --batch_size $input_batch_size \
        --input_size $input_height $input_width \
        --display_every $display_every \
        --mode validation \
        --max_workspace_size $max_workspace_size \
        --model_name $model_name \
        --output_csv $output_path

    after_record = num_record(output_path)
    if before_record == after_record:
        # failed benckmark.
        output_failed_result(output_path, model_name,
                             str(input_height) + "x" + str(input_width), "Native FP32")
    before_record = after_record

    if tensor_core_gpu:
        # TF-TRT FP32
        print("-------------------- TF-TRT FP32 model --------------------")
        output_saved_model_dir = os.path.join(output_dir, "TF_TRT_FP32")
        !TF_XLA_FLAGS=--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit python object_detection.py \
          --input_saved_model_dir $input_saved_model_dir \
          --output_saved_model_dir $output_saved_model_dir \
          --data_dir $val2017_dir \
          --annotation_path $annotations_dir \
          --batch_size $input_batch_size \
          --input_size $input_height $input_width \
          --display_every $display_every \
          --mode validation \
          --max_workspace_size $max_workspace_size \
          --minimum_segment_size $minimum_segment_size \
          --model_name $model_name \
          --output_csv $output_path \
          --use_trt \
          --precision FP32 \
          --optimize_offline

        after_record = num_record(output_path)
        if before_record == after_record:
            # failed benckmark.
            output_failed_result(output_path, model_name,
                               str(input_height) + "x" + str(input_width), "TF-TRT FP32")
        before_record = after_record

        # TF-TRT FP16
        print("start --------------------" "TF-TRT FP16" "--------------------")
        output_saved_model_dir = os.path.join(output_dir, "TF_TRT_FP16")
        !TF_XLA_FLAGS=--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit python object_detection.py \
            --input_saved_model_dir $input_saved_model_dir \
            --output_saved_model_dir $output_saved_model_dir \
            --data_dir $val2017_dir \
            --annotation_path $annotations_dir \
            --batch_size $input_batch_size \
            --input_size $input_height $input_width \
            --display_every $display_every \
            --mode validation \
            --minimum_segment_size $minimum_segment_size \
            --max_workspace_size $max_workspace_size \
            --model_name $model_name \
            --output_csv $output_path \
            --use_trt \
            --precision FP16 \
            --optimize_offline

        after_record = num_record(output_path)
        if before_record == after_record:
          # failed benckmark.
          output_failed_result(output_path, model_name, str(input_height) + "x" + str(input_width), "TF-TRT FP16")
        before_record = after_record

        # TF-TRT INT8
        print("start --------------------" "TF-TRT INT8" "--------------------")
        output_saved_model_dir = os.path.join(output_dir, "TF_TRT_INT8")
        !TF_XLA_FLAGS=--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit python object_detection.py \
            --input_saved_model_dir $input_saved_model_dir \
            --output_saved_model_dir $output_saved_model_dir \
            --data_dir $val2017_dir \
            --annotation_path $annotations_dir \
            --batch_size $input_batch_size \
            --input_size $input_height $input_width \
            --display_every $display_every \
            --mode validation \
            --minimum_segment_size $minimum_segment_size \
            --max_workspace_size $max_workspace_size \
            --model_name $model_name \
            --output_csv $output_path \
            --use_trt \
            --precision INT8 \
            --calib_data_dir $val2017_dir \
            --optimize_offline

        after_record = num_record(output_path)
        if before_record == after_record:
            # failed benckmark.
            output_failed_result(output_path, model_name,
                                 str(input_height) + "x" + str(input_width), "TF-TRT INT8")
        before_record = after_record

    # remove dowload and convert model.
    !rm -rf $tar_file_name
    !rm -rf $extract_dir

start --------------------centernet_hg104_512x512_coco17_tpu--------------------
--2021-04-19 07:33:02--  http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz
download.tensorflow.org (download.tensorflow.org) をDNSに問いあわせています... 2404:6800:4004:822::2010, 142.250.196.144
download.tensorflow.org (download.tensorflow.org)|2404:6800:4004:822::2010|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 1426099846 (1.3G) [application/x-tar]
`centernet_hg104_512x512_coco17_tpu-8.tar.gz' に保存中

centernet_hg104_512 100%[===================>]   1.33G  5.41MB/s 時間 6m 2s      

2021-04-19 07:39:05 (3.75 MB/s) - `centernet_hg104_512x512_coco17_tpu-8.tar.gz' へ保存完了 [1426099846/1426099846]

-------------------- Native FP32 model --------------------
Found the following GPUs:
   PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Benchmark arguments:
  annotation_path: ./annotations/instances_val2017.json
  batch_size: 1
